In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import tensorflow as tf
from tensorflow import keras
import gc
import random

In [ ]:
cancer = pd.read_csv("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv")

In [ ]:
cancer.info()

In [ ]:
cancer.fillna({'age': np.mean(cancer['age'])}, inplace=True)
image_paths = []
for part in ("part_1","part_2"):
    image_paths += glob.glob("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_"+part+"/*")
image_ids_n_paths = {os.path.splitext(os.path.basename(path))[0]:path for path in image_paths}
cancer['path'] = cancer['image_id'].map(image_ids_n_paths)
labels = cancer['dx'].to_frame()
cancer = cancer.drop('dx',axis=1)
labels = pd.get_dummies(labels)

In [ ]:
cancer.head()

In [ ]:
labels.head()

In [ ]:
preliminary_data = list(zip(cancer['path'],labels.values.tolist()))

In [ ]:
random.shuffle(preliminary_data)
paths, labels = zip(*preliminary_data)

In [ ]:
data = tf.data.Dataset.from_tensor_slices((list(paths),list(labels)))

In [ ]:
def final_data(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[90,120])
    image = image/255
    return image,label

In [ ]:
data = data.map(final_data).prefetch(30)

In [ ]:
train_size = round(0.8 * 10015)
val_size = round(0.1 * 10015)
test_size = 10015 - train_size - val_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size)
test = data.skip(train_size)
val = data.take(val_size)
test = data.take(test_size)

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

In [ ]:
with mirrored_strategy.scope():
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(30,(5,5),strides=(1,1),padding='valid',activation='relu',input_shape=(90,120,3)))
    model.add(tf.keras.layers.Conv2D(30,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid'))
    model.add(tf.keras.layers.Conv2D(20,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.GroupNormalization(groups=3))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid'))
    model.add(tf.keras.layers.Conv2D(10,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Normalization())
    model.add(tf.keras.layers.Dense(256,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(128,activation='relu'))
    model.add(tf.keras.layers.Dense(7,activation='softmax'))
    model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
with mirrored_strategy.scope():
    checkpoint= tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/skin_cancer_detection7.h5',save_weights_only=False,
                                                   monitor='val_accuracy',save_best_only=True,save_freq="epoch",)
    early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
    def lr_scheduler(epoch,lr,epochs=50):
        initial=1e-3
        if epoch<epochs*0.1:
            return initial
        elif epoch>epochs*0.1 and epoch<epochs*0.25:
            lr*=tf.math.exp(-0.1)
            return lr
        else:
            lr*=tf.math.exp(-0.008)
            return lr
    lr_scheduling=tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
    # Memory clear on every epoch
    class RAM_reclaim(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs=None):
            gc.collect()
    ram_reclaim=RAM_reclaim()

In [ ]:
with mirrored_strategy.scope():
    history = model.fit(train.batch(60),epochs=10,validation_data=val.batch(60),
                        callbacks=[checkpoint,early_stopping,lr_scheduling],shuffle=True)

In [ ]:
model1 = tf.keras.models.load_model("/kaggle/input/model-trained/skin_cancer_detection7.h5")

In [ ]:
predictions = model1.predict(test.batch(len(test)))

In [ ]:
predictions[0]

In [ ]:
def outputs(x):
    a = np.zeros(x.shape)
    a[np.where(x==np.max(x))] = 1
    return a

In [ ]:
for i in range(len(predictions)):
    predictions[i]=outputs(predictions[i])

In [ ]:
predictions

In [ ]:
predictions[0]

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_test = np.concatenate([y for x, y in test.batch(len(test))], axis=0)

In [ ]:
y_test

In [ ]:
accuracy_score(predictions,y_test)